# Music Style Transfer through LSTM

Here we are attempting to see whether we can transfer the style of the music by changing the notes themselves. Here, our definition of style transfer has changed. We are interested in seeing whether a network can generate music by being "inspired" from another music.

To do this, we will first train a CNN that classifies music well. Then, we will use the actiavtions from one of the layers as a feature extractor and extract features from music. 

Next step is to use the features we extracted to generate the music back. This is done by conditioning a (possibly bidirectional) LSTM using the feature vector, and then train it to generate the music that was used to extract the feature vector. We will train 2 such generators.

Last step is to see whether the generators works as a translator. We will take one music from one class and extract the features. And then feed the feature into the other class to generate music. We will see if the network picks up the styles and whether the network can be "inspired" by the music

In [1]:
import os
import mido
import keras
import numpy as np
import sklearn.model_selection as ms
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
set_session(session)

In [3]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [4]:
X = np.load('matricies/notes.npy')
Y = np.load('matricies/labels.npy')

For this task we will toss the first dimension since it does not add additional information

In [5]:
X = X[:,:,:,1]
print(X.shape)

(7034, 480, 59)


In [8]:
X_train, X_test, Y_train, Y_test = ms.train_test_split(X, Y, test_size=0.1, random_state=123)
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(6330, 480, 59) (704, 480, 59) (6330, 1) (704, 1)


## CNN encoding

In this section we first try to train a CNN that classifies music well. Here we will use a simple structure of

1. Conv
2. Dense
...


In [9]:
def model1(input_shape):
    X_input = keras.layers.Input(input_shape)
    X = X_input
    X = keras.layers.Conv1D(filters=4, kernel_size=32, padding='same', name='Conv0',
                            dilation_rate=4,
                            kernel_initializer=keras.initializers.glorot_normal(seed=None),
                            bias_initializer=keras.initializers.glorot_normal(seed=None),
                            data_format="channels_last")(X)
    X = keras.layers.AveragePooling1D(4)(X)
    X = keras.layers.BatchNormalization(axis = 2, name = 'bn0')(X)
    X = keras.layers.Activation('relu')(X)
    print(X.shape)
    
#     X = keras.layers.Reshape((250, -1))(X)
    X = keras.layers.Conv1D(filters=16, kernel_size=16, padding='valid', name='Conv1',
                            dilation_rate=2,
                            kernel_initializer=keras.initializers.glorot_normal(seed=None),
                            bias_initializer=keras.initializers.glorot_normal(seed=None),
                            data_format="channels_last")(X)
    X = keras.layers.AveragePooling1D(2)(X)
    X = keras.layers.BatchNormalization(axis = 2, name = 'bn1')(X)
    X = keras.layers.Activation('relu')(X)
    print(X.shape)
    X = keras.layers.Flatten()(X)
#     X = keras.layers.Dense(100, activation='sigmoid')(X)
#     X = keras.layers.Dropout(0.5)(X)    
    X = keras.layers.Dense(50, activation='sigmoid')(X)
    X = keras.layers.Dropout(0.5)(X)    
#     X = keras.layers.Dense(10, activation='sigmoid')(X)
    X = keras.layers.Dense(1, activation='sigmoid')(X)
    model = keras.models.Model(inputs=X_input, outputs=X, name='basic')
    return model

In [10]:
m = model1(X.shape[1:])
m.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy'])

(?, 120, 4)
(?, 45, 16)


In [11]:
print(m.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 480, 59)           0         
_________________________________________________________________
Conv0 (Conv1D)               (None, 480, 4)            7556      
_________________________________________________________________
average_pooling1d_1 (Average (None, 120, 4)            0         
_________________________________________________________________
bn0 (BatchNormalization)     (None, 120, 4)            16        
_________________________________________________________________
activation_1 (Activation)    (None, 120, 4)            0         
_________________________________________________________________
Conv1 (Conv1D)               (None, 90, 16)            1040      
_________________________________________________________________
average_pooling1d_2 (Average (None, 45, 16)            0         
__________

In [14]:
m.fit(X_train, Y_train, epochs=4, batch_size=128)

Epoch 1/4
6330/6330 [==============================] - 6s 965us/step - loss: 0.1754 - binary_accuracy: 0.9374
Epoch 2/4
6330/6330 [==============================] - 6s 966us/step - loss: 0.1598 - binary_accuracy: 0.9408
Epoch 3/4
6330/6330 [==============================] - 6s 964us/step - loss: 0.1447 - binary_accuracy: 0.9479
Epoch 4/4
6330/6330 [==============================] - 6s 966us/step - loss: 0.1307 - binary_accuracy: 0.9521


In [15]:
preds = m.evaluate(X_test, Y_test, batch_size=32)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

704/704 [==============================] - 0s 627us/step
Loss = 0.21432351574979044
Test Accuracy = 0.9119318181818182


In [16]:
m.save('cnn.h5')

In [6]:
m = keras.models.load_model('cnn.h5')

### Feature Extraction

Below is code for feature extraction.

In [7]:
m.layers[9]

In [8]:
def extract_all_features(X, model, layer_nums, batch_num):
    """Extract all the features by taking out the activation output of the layers specified in layer_nums
    
        X : the data that we are trying to extract features from
        model: the model that we are using to extract the features
        layer_nums: the layer number that we want to use as feature extractors
    """
    outputs = [model.layers[layer].output for layer in layer_nums]
    f = keras.backend.function([model.input, keras.backend.learning_phase()], outputs)
    ## split up into smaller chunks
    layer_outs = []
    m = X.shape[0]
    for i in range(batch_num + 1):
        layer_outs += [f([X[(m // batch_num * i):(m // batch_num * (i+1)), :, :], 0.])]
    result = []
    for i in range(len(layer_nums)):
        layer_outs2 = []
        for batch in range(batch_num):
            activations = layer_outs[batch][i]
            activations = activations.reshape(activations.shape[0], -1)
            layer_outs2 += [activations]
        result += [np.vstack(np.array(layer_outs2))]
    return result

In [9]:
classical_features = extract_all_features(X[(Y == 1).ravel(), :, :], m, [9, 10], 160)

In [10]:
jazz_features = extract_all_features(X[(Y == 0).ravel(), :, :], m, [9, 10], 160)

## LSTM 

Here we will train 2 (bidirectional) LSTMs. One for generating classical music and one for generating jazz music. The way we condition the LSTMs is to have a Dense layer that transforms our feature vectors into the shape of the cell state of the LSTM, and then we will use this cell state as the initial cell state of the LSTM, and train it to generate the original MIDI matrix. 

The input to the model is a feature vector. It will then have a dense layer that transforms it into the cell state shape. Then for every time stamp it will generate some music, and the output should be a music.

First we need to construct the new X and Ys. The X will be the same but with the last layer stacked on top of each other insead. Y will be the same as X but all values shifted left by 1 timestamp

In [11]:
gen_T = 160
m_ckeep = classical_features[1].shape[0]
m_jkeep = jazz_features[1].shape[0]
X2_classical = X[(Y==1).ravel(), :,:][:m_ckeep,:gen_T,:]
X2_jazz = X[(Y==0).ravel(), :m_jkeep,:][:m_jkeep,:gen_T,:]
Y2_classical = np.zeros(X2_classical.shape, dtype=np.int32)
Y2_classical[:-1,:,:] = X2_classical[1:,:,:]
Y2_jazz = np.zeros(X2_jazz.shape, dtype=np.int32)
Y2_jazz[:-1,:,:] = X2_jazz[1:,:,:]

Now we need to extract the features we will use to create the beautiful piece of music. 

In [12]:
X2_classical.shape

(4160, 160, 59)

The following code is inspired and borrowed from Coursera Sequence Model Module/Improvise a Jazz Solo with an LSTM Network - v3 programming assignment

In [13]:
n_a = 64
n_values = 59
LSTM_cell = keras.layers.LSTM(n_a, return_state = True)
feature_densor = keras.layers.Dense(n_a, activation='sigmoid')
# on_off_densor = keras.layers.Dense(n_values, activation='sigmoid')
sustain_densor = keras.layers.Dense(n_values, activation='sigmoid')
reshapor = keras.layers.Reshape((1, n_values))
concatenator = keras.layers.Concatenate(axis=-1)

In [14]:
def lstm(feature_shape, Tx, n_channel):
    feature_input = keras.layers.Input(feature_shape, name='feature_inp')
    music_input = keras.layers.Input((Tx, n_channel), name='X')
    a0 = keras.layers.Input((n_a,), name='a0')
    
#     feature = keras.layers.Flatten()(feature_input)
    c0 = feature_densor(feature_input)
    X = music_input
    a = a0
    c = c0
    outputs = []
    for t in range(Tx):
        if t % 200 == 0:
            print(t)
        x = keras.layers.Lambda(lambda x: X[:, t, :])(X)
        x = reshapor(x)
        a, _, c = LSTM_cell(x, initial_state=[a, c])
#         on_off_out = on_off_densor(a)
        sustain_out = sustain_densor(a)
#         outputs += [concatenator([on_off_out, sustain_out])]
        outputs += [sustain_out]
    return keras.models.Model(inputs=[feature_input, music_input, a0], outputs=outputs)
    
    

In [15]:
lstm_model = lstm((classical_features[1].shape[1], ), gen_T, X.shape[2])
lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy'])

0


In [ ]:
X2_classical[1].shape

(160, 59)

In [ ]:
mini_size = 200
a0 = np.zeros((mini_size, n_a))
lstm_model.fit([classical_features[1][:mini_size:,:], X2_classical[:mini_size,:], a0],
               list(Y2_classical[:mini_size,:].transpose((1, 0, 2))), epochs=500) 

Epoch 1/500
200/200 [==============================] - 130s 650ms/step - loss: 109.9448 - dense_2_loss: 0.6867 - dense_2_binary_accuracy: 0.4434 - dense_2_binary_accuracy_1: 0.4741 - dense_2_binary_accuracy_2: 0.4940 - dense_2_binary_accuracy_3: 0.5177 - dense_2_binary_accuracy_4: 0.5311 - dense_2_binary_accuracy_5: 0.5479 - dense_2_binary_accuracy_6: 0.5525 - dense_2_binary_accuracy_7: 0.5636 - dense_2_binary_accuracy_8: 0.5611 - dense_2_binary_accuracy_9: 0.5653 - dense_2_binary_accuracy_10: 0.5698 - dense_2_binary_accuracy_11: 0.5712 - dense_2_binary_accuracy_12: 0.5727 - dense_2_binary_accuracy_13: 0.5769 - dense_2_binary_accuracy_14: 0.5770 - dense_2_binary_accuracy_15: 0.5733 - dense_2_binary_accuracy_16: 0.5705 - dense_2_binary_accuracy_17: 0.5736 - dense_2_binary_accuracy_18: 0.5757 - dense_2_binary_accuracy_19: 0.5769 - dense_2_binary_accuracy_20: 0.5753 - dense_2_binary_accuracy_21: 0.5750 - dense_2_binary_accuracy_22: 0.5768 - dense_2_binary_accuracy_23: 0.5732 - dense_2_bin

 64/200 [========>.....................] - ETA: 2s - loss: 107.3611 - dense_2_loss: 0.6696 - dense_2_binary_accuracy: 0.4862 - dense_2_binary_accuracy_1: 0.5260 - dense_2_binary_accuracy_2: 0.5742 - dense_2_binary_accuracy_3: 0.6112 - dense_2_binary_accuracy_4: 0.6412 - dense_2_binary_accuracy_5: 0.6637 - dense_2_binary_accuracy_6: 0.6727 - dense_2_binary_accuracy_7: 0.6894 - dense_2_binary_accuracy_8: 0.6904 - dense_2_binary_accuracy_9: 0.6862 - dense_2_binary_accuracy_10: 0.6954 - dense_2_binary_accuracy_11: 0.6939 - dense_2_binary_accuracy_12: 0.6941 - dense_2_binary_accuracy_13: 0.6994 - dense_2_binary_accuracy_14: 0.6992 - dense_2_binary_accuracy_15: 0.6978 - dense_2_binary_accuracy_16: 0.6986 - dense_2_binary_accuracy_17: 0.7055 - dense_2_binary_accuracy_18: 0.7097 - dense_2_binary_accuracy_19: 0.7105 - dense_2_binary_accuracy_20: 0.7090 - dense_2_binary_accuracy_21: 0.7082 - dense_2_binary_accuracy_22: 0.7010 - dense_2_binary_accuracy_23: 0.7076 - dense_2_binary_accuracy_24: 0.6

 96/200 [=============>................] - ETA: 1s - loss: 106.8543 - dense_2_loss: 0.6672 - dense_2_binary_accuracy: 0.4936 - dense_2_binary_accuracy_1: 0.5358 - dense_2_binary_accuracy_2: 0.5890 - dense_2_binary_accuracy_3: 0.6329 - dense_2_binary_accuracy_4: 0.6598 - dense_2_binary_accuracy_5: 0.6829 - dense_2_binary_accuracy_6: 0.6901 - dense_2_binary_accuracy_7: 0.6992 - dense_2_binary_accuracy_8: 0.6999 - dense_2_binary_accuracy_9: 0.6974 - dense_2_binary_accuracy_10: 0.7057 - dense_2_binary_accuracy_11: 0.7030 - dense_2_binary_accuracy_12: 0.7034 - dense_2_binary_accuracy_13: 0.7064 - dense_2_binary_accuracy_14: 0.7064 - dense_2_binary_accuracy_15: 0.7052 - dense_2_binary_accuracy_16: 0.7050 - dense_2_binary_accuracy_17: 0.7135 - dense_2_binary_accuracy_18: 0.7140 - dense_2_binary_accuracy_19: 0.7170 - dense_2_binary_accuracy_20: 0.7157 - dense_2_binary_accuracy_21: 0.7140 - dense_2_binary_accuracy_22: 0.7120 - dense_2_binary_accuracy_23: 0.7163 - dense_2_binary_accuracy_24: 0.7

128/200 [==================>...........] - ETA: 1s - loss: 106.5045 - dense_2_loss: 0.6650 - dense_2_binary_accuracy: 0.5016 - dense_2_binary_accuracy_1: 0.5453 - dense_2_binary_accuracy_2: 0.6014 - dense_2_binary_accuracy_3: 0.6447 - dense_2_binary_accuracy_4: 0.6709 - dense_2_binary_accuracy_5: 0.6916 - dense_2_binary_accuracy_6: 0.6972 - dense_2_binary_accuracy_7: 0.7054 - dense_2_binary_accuracy_8: 0.7060 - dense_2_binary_accuracy_9: 0.7055 - dense_2_binary_accuracy_10: 0.7128 - dense_2_binary_accuracy_11: 0.7109 - dense_2_binary_accuracy_12: 0.7121 - dense_2_binary_accuracy_13: 0.7132 - dense_2_binary_accuracy_14: 0.7149 - dense_2_binary_accuracy_15: 0.7124 - dense_2_binary_accuracy_16: 0.7108 - dense_2_binary_accuracy_17: 0.7178 - dense_2_binary_accuracy_18: 0.7178 - dense_2_binary_accuracy_19: 0.7214 - dense_2_binary_accuracy_20: 0.7215 - dense_2_binary_accuracy_21: 0.7202 - dense_2_binary_accuracy_22: 0.7180 - dense_2_binary_accuracy_23: 0.7203 - dense_2_binary_accuracy_24: 0.7

160/200 [=======================>......] - ETA: 0s - loss: 106.0139 - dense_2_loss: 0.6619 - dense_2_binary_accuracy: 0.5056 - dense_2_binary_accuracy_1: 0.5541 - dense_2_binary_accuracy_2: 0.6102 - dense_2_binary_accuracy_3: 0.6538 - dense_2_binary_accuracy_4: 0.6829 - dense_2_binary_accuracy_5: 0.7047 - dense_2_binary_accuracy_6: 0.7106 - dense_2_binary_accuracy_7: 0.7192 - dense_2_binary_accuracy_8: 0.7222 - dense_2_binary_accuracy_9: 0.7255 - dense_2_binary_accuracy_10: 0.7304 - dense_2_binary_accuracy_11: 0.7297 - dense_2_binary_accuracy_12: 0.7294 - dense_2_binary_accuracy_13: 0.7296 - dense_2_binary_accuracy_14: 0.7300 - dense_2_binary_accuracy_15: 0.7251 - dense_2_binary_accuracy_16: 0.7229 - dense_2_binary_accuracy_17: 0.7305 - dense_2_binary_accuracy_18: 0.7305 - dense_2_binary_accuracy_19: 0.7321 - dense_2_binary_accuracy_20: 0.7338 - dense_2_binary_accuracy_21: 0.7327 - dense_2_binary_accuracy_22: 0.7314 - dense_2_binary_accuracy_23: 0.7341 - dense_2_binary_accuracy_24: 0.7

...

128/200 [==================>...........] - ETA: 1s - loss: 29.5409 - dense_2_loss: 0.1764 - dense_2_binary_accuracy: 0.9346 - dense_2_binary_accuracy_1: 0.9525 - dense_2_binary_accuracy_2: 0.9507 - dense_2_binary_accuracy_3: 0.9486 - dense_2_binary_accuracy_4: 0.9509 - dense_2_binary_accuracy_5: 0.9543 - dense_2_binary_accuracy_6: 0.9521 - dense_2_binary_accuracy_7: 0.9503 - dense_2_binary_accuracy_8: 0.9481 - dense_2_binary_accuracy_9: 0.9526 - dense_2_binary_accuracy_10: 0.9506 - dense_2_binary_accuracy_11: 0.9477 - dense_2_binary_accuracy_12: 0.9492 - dense_2_binary_accuracy_13: 0.9490 - dense_2_binary_accuracy_14: 0.9488 - dense_2_binary_accuracy_15: 0.9484 - dense_2_binary_accuracy_16: 0.9472 - dense_2_binary_accuracy_17: 0.9514 - dense_2_binary_accuracy_18: 0.9485 - dense_2_binary_accuracy_19: 0.9509 - dense_2_binary_accuracy_20: 0.9493 - dense_2_binary_accuracy_21: 0.9497 - dense_2_binary_accuracy_22: 0.9502 - dense_2_binary_accuracy_23: 0.9495 - dense_2_binary_accuracy_24: 0.95

160/200 [=======================>......] - ETA: 0s - loss: 29.3556 - dense_2_loss: 0.1763 - dense_2_binary_accuracy: 0.9360 - dense_2_binary_accuracy_1: 0.9531 - dense_2_binary_accuracy_2: 0.9512 - dense_2_binary_accuracy_3: 0.9510 - dense_2_binary_accuracy_4: 0.9519 - dense_2_binary_accuracy_5: 0.9555 - dense_2_binary_accuracy_6: 0.9526 - dense_2_binary_accuracy_7: 0.9522 - dense_2_binary_accuracy_8: 0.9501 - dense_2_binary_accuracy_9: 0.9538 - dense_2_binary_accuracy_10: 0.9533 - dense_2_binary_accuracy_11: 0.9497 - dense_2_binary_accuracy_12: 0.9502 - dense_2_binary_accuracy_13: 0.9500 - dense_2_binary_accuracy_14: 0.9510 - dense_2_binary_accuracy_15: 0.9498 - dense_2_binary_accuracy_16: 0.9474 - dense_2_binary_accuracy_17: 0.9520 - dense_2_binary_accuracy_18: 0.9489 - dense_2_binary_accuracy_19: 0.9500 - dense_2_binary_accuracy_20: 0.9493 - dense_2_binary_accuracy_21: 0.9504 - dense_2_binary_accuracy_22: 0.9500 - dense_2_binary_accuracy_23: 0.9483 - dense_2_binary_accuracy_24: 0.95

192/200 [===========================>..] - ETA: 0s - loss: 29.2883 - dense_2_loss: 0.1755 - dense_2_binary_accuracy: 0.9357 - dense_2_binary_accuracy_1: 0.9529 - dense_2_binary_accuracy_2: 0.9522 - dense_2_binary_accuracy_3: 0.9518 - dense_2_binary_accuracy_4: 0.9524 - dense_2_binary_accuracy_5: 0.9547 - dense_2_binary_accuracy_6: 0.9514 - dense_2_binary_accuracy_7: 0.9522 - dense_2_binary_accuracy_8: 0.9499 - dense_2_binary_accuracy_9: 0.9531 - dense_2_binary_accuracy_10: 0.9529 - dense_2_binary_accuracy_11: 0.9498 - dense_2_binary_accuracy_12: 0.9507 - dense_2_binary_accuracy_13: 0.9508 - dense_2_binary_accuracy_14: 0.9520 - dense_2_binary_accuracy_15: 0.9504 - dense_2_binary_accuracy_16: 0.9484 - dense_2_binary_accuracy_17: 0.9516 - dense_2_binary_accuracy_18: 0.9494 - dense_2_binary_accuracy_19: 0.9491 - dense_2_binary_accuracy_20: 0.9485 - dense_2_binary_accuracy_21: 0.9488 - dense_2_binary_accuracy_22: 0.9484 - dense_2_binary_accuracy_23: 0.9471 - dense_2_binary_accuracy_24: 0.95

200/200 [==============================] - 4s 22ms/step - loss: 29.1429 - dense_2_loss: 0.1765 - dense_2_binary_accuracy: 0.9366 - dense_2_binary_accuracy_1: 0.9538 - dense_2_binary_accuracy_2: 0.9533 - dense_2_binary_accuracy_3: 0.9527 - dense_2_binary_accuracy_4: 0.9533 - dense_2_binary_accuracy_5: 0.9553 - dense_2_binary_accuracy_6: 0.9522 - dense_2_binary_accuracy_7: 0.9529 - dense_2_binary_accuracy_8: 0.9505 - dense_2_binary_accuracy_9: 0.9539 - dense_2_binary_accuracy_10: 0.9539 - dense_2_binary_accuracy_11: 0.9507 - dense_2_binary_accuracy_12: 0.9516 - dense_2_binary_accuracy_13: 0.9519 - dense_2_binary_accuracy_14: 0.9529 - dense_2_binary_accuracy_15: 0.9511 - dense_2_binary_accuracy_16: 0.9492 - dense_2_binary_accuracy_17: 0.9525 - dense_2_binary_accuracy_18: 0.9501 - dense_2_binary_accuracy_19: 0.9497 - dense_2_binary_accuracy_20: 0.9492 - dense_2_binary_accuracy_21: 0.9495 - dense_2_binary_accuracy_22: 0.9492 - dense_2_binary_accuracy_23: 0.9479 - dense_2_binary_accuracy_24:

Epoch 9/500
 32/200 [===>..........................] - ETA: 3s - loss: 29.0434 - dense_2_loss: 0.1841 - dense_2_binary_accuracy: 0.9428 - dense_2_binary_accuracy_1: 0.9571 - dense_2_binary_accuracy_2: 0.9566 - dense_2_binary_accuracy_3: 0.9544 - dense_2_binary_accuracy_4: 0.9544 - dense_2_binary_accuracy_5: 0.9571 - dense_2_binary_accuracy_6: 0.9518 - dense_2_binary_accuracy_7: 0.9544 - dense_2_binary_accuracy_8: 0.9523 - dense_2_binary_accuracy_9: 0.9534 - dense_2_binary_accuracy_10: 0.9518 - dense_2_binary_accuracy_11: 0.9502 - dense_2_binary_accuracy_12: 0.9518 - dense_2_binary_accuracy_13: 0.9465 - dense_2_binary_accuracy_14: 0.9513 - dense_2_binary_accuracy_15: 0.9444 - dense_2_binary_accuracy_16: 0.9502 - dense_2_binary_accuracy_17: 0.9476 - dense_2_binary_accuracy_18: 0.9465 - dense_2_binary_accuracy_19: 0.9417 - dense_2_binary_accuracy_20: 0.9454 - dense_2_binary_accuracy_21: 0.9433 - dense_2_binary_accuracy_22: 0.9439 - dense_2_binary_accuracy_23: 0.9433 - dense_2_binary_accur

### Generate Music with LSTM

Now we will sample the LSTM and densor we just trained to generate some cool music

In [ ]:
lstm_model.reset_states()

In [37]:
def lstm_generate(LSTM_cell, sustain_densor, feature_shape, n_values, n_a, Ty=200):
    x0 = keras.layers.Input((1, n_values), name='x0')
    feature_input = keras.layers.Input(feature_shape, name='feature_inp')
    a0 = keras.layers.Input((n_a,), name='a0')
    
    c0 = feature_densor(feature_input)
    a = a0
    c = c0
    x = x0
    outputs = []
    for t in range(Ty):
        a, _, c = LSTM_cell(x, initial_state=[a, c])
        out = sustain_densor(a)
        out = reshapor(out)
        outputs += [out]
        x = out
    return keras.models.Model(inputs=[feature_input, x0, a0], outputs=outputs)

In [38]:
generate_model = lstm_generate(LSTM_cell, sustain_densor, (classical_features[1].shape[1],), n_values, n_a)

ValueError: Layer lstm_1 expects 7 inputs, but it received 3 input tensors. Input received: [<tf.Tensor 'x0_2:0' shape=(?, 1, 59) dtype=float32>, <tf.Tensor 'a0_4:0' shape=(?, 64) dtype=float32>, <tf.Tensor 'dense_3_4/Sigmoid:0' shape=(?, 64) dtype=float32>]

In [ ]:
feature_input = classical_features[1][:1,:]
x0 = np.zeros((1, 1, n_values*2))
a0 = np.zeros((1, n_a,))
print(feature_input.shape, x0.shape,a0.shape)

In [ ]:
pred = generate_model.predict([feature_input, x0, a0])

In [ ]:
song = np.array(pred).reshape(-1, n_values*2)
song = song > 0.1
print(song.shape)

In [ ]:
plt.figure()
plt.imshow(song)

In [ ]:
target_ticks_per_beat = 16
def generate(notes_mat, velocity_mat, info):
    new_midi = mido.MidiFile(type=0)
    new_midi.ticks_per_beat = target_ticks_per_beat
    track = mido.MidiTrack()
    new_midi.tracks.append(track)

    track.append(mido.MetaMessage('set_tempo', tempo=info['tempo'], time=0))
    track.append(mido.MetaMessage('time_signature', numerator=info['numerator'], denominator=info['denominator'], 
                             clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0))
    track.append(mido.MetaMessage('track_name', name='Test Track', time=0))

    T, N, D = notes_mat.shape
    notes_on = [False] * N
    prev_event_t = 0
    for t in range(T):
        for n in range(N-2):
            if notes_mat[t, n, 0] > 0:
                velocity = velocity_mat[t, n]
                track.append(mido.Message('note_on', note=n, velocity=velocity, time=t-prev_event_t))
                prev_event_t = t
                notes_on[n] = True
            if notes_on[n] and notes_mat[t, n, 1] == 0:
                track.append(mido.Message('note_on', note=n, velocity=0, time=t-prev_event_t))
                prev_event_t = t
                notes_on[n] = False
        if notes_mat[t, N-2, 0] == 1:
            track.append(mido.Message('control_change', control=64, value=127, time=t-prev_event_t))
            prev_event_t = t
            notes_on[N-2] = True
        if notes_mat[t, N-1, 0] == 1:
            track.append(mido.Message('control_change', control=67, value=127, time=t-prev_event_t))
            prev_event_t = t
            notes_on[N-2] = True
        if notes_on[N-2] and notes_mat[t, N-2, 1] == 0:
            track.append(mido.Message('control_change', control=64, value=0, time=t-prev_event_t))
            prev_event_t = t
            notes_on[N-2] = False
        if notes_on[N-1] and notes_mat[t, N-1, 1] == 0:
            track.append(mido.Message('control_change', control=67, value=0, time=t-prev_event_t))
            prev_event_t = t
            notes_on[N-1] = False
    return new_midi

In [ ]:
info = {'tempo':50000, 'numerator':4, 'denominator':4}
new_midi = generate((song > 0.02).reshape(200, 130,2) , np.ones(song.shape, dtype=np.int8) * 64, info)
new_midi.save('temp.midi')